In [1]:
## 导入必要的包
#encoding=utf-8
import pandas as pd
import numpy as np

# 画图
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# 导入数据
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
gender = pd.read_csv('data/gender_submission.csv')


#### 从描述数据开始分析


#### 问题1： 有多少训练数据


In [11]:
len(train)

891

#### 问题2： 训练数据包括哪些属性？

In [13]:
train.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')

#### 问题3： 这些属性中，哪些是数值连续型，哪些是离散型？

In [14]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


- 连续型：Fare
- 离散型：PassengerId, Survived, Pclass, Sex, Age, SibSp, Parch, Embarked
- 字符型：Name，Sex， Ticket，Cabin，Embarked



In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


可以看出有缺失值，下面简单统计一下：

In [22]:
for col in train.columns:
    dd = train[col].isnull().sum()
    if dd >0:
        print u'%s 有 %s 条缺失值' % (col,  train[col].isnull().sum())

Age 有 177 条缺失值
Cabin 有 687 条缺失值
Embarked 有 2 条缺失值


#### 问题4： 统计上来看，数据有什么特征

In [23]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


从上表中作出如下解读：
- Survived 有2种取值，0-表示死亡，1-表示存活，均值就表示存活率，训练数据中有38%人存活
- 训练数据中，年纪最大是80岁，最小的还没有半岁，绝大多数人在38岁以下（>75%）
- 大多数人没有跟兄弟或者伴侣一起上船 (>75%)
- 大多数人没有跟父母和子女一起上船 (>75%)
- Fare 值差异很大，最小是0最大512

显示所有为离散型数据列情况

In [24]:
train.describe(include=['O']) # 字母欧O

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Graham, Mr. George Edward",male,CA. 2343,C23 C25 C27,S
freq,1,577,7,4,644


可以看出：
- 男性成员有577名
- 票据有 891-681=210张重复，占~24%
- 船舱有相同的数据，表示有乘客共享船舱
- Embarked 有3个可能取值，其中 S 占大多数


### 相关性分析 Correlation analysis

根据以上对数据的简单分析，可以看到数据分布以及缺失值情况。下面要从特征中选择一些对生存率影响较大的，单独拉出来分析，这会影响到后续使用的数据和建模。


- PassengerId 乘客标识而已，对生存率没有影响，可删
- Pclass 舱位等级，直觉有关（更好的舱位，设施更好，安全通道应该也不错），保留
- Name 姓名，叫什么名字跟存活应该没有毛线关系，删
- Sex 性别，据说女士和儿童优先，有关
- Age 同上一条


In [33]:
train[['Pclass', 'Survived']].groupby('Pclass', as_index=False).mean().sort_values('Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [34]:
train[['Sex', 'Survived']].groupby('Sex', as_index=False).mean().sort_values('Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [35]:
train[['Embarked', 'Survived']].groupby('Embarked', as_index=False).mean().sort_values('Survived', ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957
